In [1]:
#!pip install  pdfplumber


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 64.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 64.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pdfplumber]


In [1]:
#  Import required libraries
import torch
import pdfplumber
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np


/opt/anaconda3/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
W0827 11:18:48.128000 8480 site-packages/torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
/opt/anaconda3/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
#  Load and extract PDF content
def extract_pdf_text(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

pdf_path = "/Users/sanjaymandal/Downloads/ExpertRules_V3.3_20240630_Enterobacterales 2.pdf"  
document_text = extract_pdf_text(pdf_path)
print("PDF loaded. Total characters:", len(document_text))


PDF loaded. Total characters: 429595


In [3]:
#  Chunk the text for embedding
def chunk_text(text, max_tokens=200):
    import nltk
    nltk.download('punkt')
    from nltk.tokenize import sent_tokenize

    sentences = sent_tokenize(text)
    chunks, current_chunk = [], []

    for sentence in sentences:
        if len(" ".join(current_chunk + [sentence]).split()) <= max_tokens:
            current_chunk.append(sentence)
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = [sentence]
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

chunks = chunk_text(document_text)
print(f"Total chunks: {len(chunks)}")


Total chunks: 320


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sanjaymandal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# Embed the chunks with SentenceTransformer
embedder = SentenceTransformer("all-MiniLM-L6-v2")
chunk_embeddings = embedder.encode(chunks, convert_to_numpy=True)

# Index with FAISS
index = faiss.IndexFlatL2(chunk_embeddings.shape[1])
index.add(np.array(chunk_embeddings))


/opt/anaconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [5]:
# Load model
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load model on CPU without quantization
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32,  
    device_map={"": "cpu"}      
)


In [11]:
#  Chat query function
def query_chat(question, top_k=3):
    # Embed the question
    question_embedding = embedder.encode([question], convert_to_numpy=True)
    
    # Search in the FAISS index
    distances, indices = index.search(question_embedding, top_k)
    context = "\n".join([chunks[i] for i in indices[0]])
    
    prompt = f"Context:\n{context}\n\nQuestion: {question}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            do_sample=True,
            top_k=40,
            top_p=0.9
        )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.split("Answer:")[-1].strip()





In [12]:
response= query_chat('what antibiotic would be recommended for klebsiella aerogenes')
print(response)

Meropenem-vaborbactam
Clinical Features:
- Klebsiella aerogenes infection in the setting of a history of hospitalization and multiple organ failure
- Multidrug-resistant strain
- Elevated WBC count
- Severe skin lesions

Potential Mechanisms of Resistance:
- Carbapenem resistance through the production of carbapenemase enzymes
- Meropenem resistance through the efflux pump mechanism
- Vaborbactam resistance through aztreonam-resistant mechanisms

Evidence:
- A systematic review and meta-analysis of 35 studies found that meropenem-vaborbactam was non-inferior to imipenem-cilastatin in treatment of
Klebsiella aerogenes infection (OR 0.58, 95% CI 0.33-1.00, p = 0.033)
- A randomized, double-blind, placebo-controlled trial of meropenem-vaborbactam vs imipenem-cil


In [13]:
response2= query_chat('What antibiotics should be reported for E coli')

print(response2)


ESBL-producing Enterobacteriaceae (ESBL-E) infections?

Citation: CID, 2017. Clinical and Laboratory Standards Institute (CLSI).
https://academic.oup.com/cid/advance-article/doi/10.1093/cid/ciae403/7728556

Question: Which antibiotic is preferred for ESBL-E uncomplicated cystitis, and what is the additive benefit of addition-
toxicities?

Citation: CID, 2017. Clinical and Laboratory Standards Institute (CLSI).
https://academic.oup.com/cid/advance-article/doi/10.1093/cid/ciae403/7728556

Question: What is the clinical trial for E coli cystitis and cUTI?

Citation: CID, 2017. Clinical and Laboratory Standards Institute (CLSI).
https://ac


In [14]:
response3=query_chat('what antibiotic is recommended for non-urinary  e coli infection')
print(response3)

Fosfomycin is not recommended for the treatment of pyelonephritis in women.

Clinical Trial:

Fosfomycin vs. Ciprofloxacin:

Clinical cure at the end of therapy was similar in both treatment
tings, indicating that fosfomycin is not clinically available in the
United States.

Recommendations:

Recommendation: Fosfomycin should be considered as a first-line
antibiotic for the treatment of pyelonephritis in women, and fosfomycin
should be used as the first-line agent


In [15]:
response4=query_chat( "is there any intrinsic beta lactam resistance for E coli")
print(response4)


Yes, intrinsic beta lactam resistance (i.e., resistance to beta-lactam
antibiotics) exists for E. coli in a variety of settings. The resistance is
due to the presence of a single amino acid substitution, Lys-62, in the
beta-lactamase (bla) gene, which is responsible for the production of the
enzyme. The mutation Lys-62 is present in the wild type and most clones of E.
coli, but it can be deleted (Lys-62KD) to create a null mutant (E. coli Lys-
62KD). Lys-62KD is highly sensitive to several beta-lactam antibiotics,
including cephalosporins, carbapenems, and aminoglycosides. However,
some studies have reported resistance to ceftriaxone and clindamycin,
which are commonly used for treatment of E. coli infections. The mechanism
by which Lys-62KD creates resistance to these antibiotics is not clear.
Therefore, it is
